In [2]:
import numpy as np
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Flatten, Dense, Dropout
import re

In [3]:
# 쿠팡 쇼핑 리뷰 데이터셋 다운로드
data = pd.read_excel('dataset.xlsx')

reviews = data['리뷰']
ratings = data['평점']


In [4]:
# 데이터 전처리
tokenizer = Tokenizer()
tokenizer.fit_on_texts(reviews)
vocab_size = len(tokenizer.word_index) + 1


x_sequences = tokenizer.texts_to_sequences(reviews)                # 텍스트 데이터를 숫자 시퀀스로 변환
x_padded = pad_sequences(x_sequences, maxlen=100, padding='post') # 시퀀스 데이터의 길이를 맞추기 위해 패딩 추가

y = np.array(ratings.apply(lambda x: 1 if x >= 3 else 0))       # 평점을 기준으로 긍정(1)과 부정(0)을 나타내는 이진 클래스 생성

x_train, x_test, y_train, y_test = train_test_split(x_padded, y, test_size=0.2, random_state=42)

print("긍정(1) or 부정(0) :",y[0])
print(reviews[0])
print(x_sequences[0])


긍정(1) or 부정(0) : 1
생각보다 길고 원단 톡톡해요근데 건조기 돌리니 좀 줄어들었어요만족해서 하나 더 규매했어요
[90, 1530, 3180, 9768, 9769, 5627, 9, 9770, 105, 10, 9771]


In [5]:
# 모델 구성
model = Sequential()
model.add(Embedding(vocab_size, 64, input_length=100))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 64)           3069248   
                                                                 
 flatten (Flatten)           (None, 6400)              0         
                                                                 
 dense (Dense)               (None, 64)                409664    
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 3,478,977
Trainable params: 3,478,977
Non-trainable params: 0
_________________________________________________________________


In [6]:
# 모델 학습
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

history = model.fit(x_train, y_train, batch_size=64, epochs=20, verbose=1, validation_data=(x_test, y_test))

Epoch 1/20
65/65 [==============================] - 6s 74ms/step - loss: 0.6911 - accuracy: 0.5318 - val_loss: 0.6843 - val_accuracy: 0.6044
Epoch 2/20
65/65 [==============================] - 5s 77ms/step - loss: 0.5831 - accuracy: 0.7374 - val_loss: 0.4678 - val_accuracy: 0.7843
Epoch 3/20
65/65 [==============================] - 5s 81ms/step - loss: 0.1578 - accuracy: 0.9511 - val_loss: 0.3777 - val_accuracy: 0.8308
Epoch 4/20
65/65 [==============================] - 6s 85ms/step - loss: 0.0399 - accuracy: 0.9903 - val_loss: 0.4097 - val_accuracy: 0.8337
Epoch 5/20
65/65 [==============================] - 5s 84ms/step - loss: 0.0151 - accuracy: 0.9981 - val_loss: 0.4123 - val_accuracy: 0.8308
Epoch 6/20
65/65 [==============================] - 5s 84ms/step - loss: 0.0080 - accuracy: 0.9985 - val_loss: 0.4382 - val_accuracy: 0.8279
Epoch 7/20
65/65 [==============================] - 5s 82ms/step - loss: 0.0059 - accuracy: 0.9988 - val_loss: 0.4533 - val_accuracy: 0.8298
Epoch 8/20
65

In [8]:
# 모델 서비스 구현
review = "배송이 느려서 별로였어요"

review = re.sub("[^0-9ㄱ-ㅎㅏ-ㅣ가-힣 ]", "", review).lower()
review_encoding = tokenizer.texts_to_sequences([review])
review_padded = pad_sequences(review_encoding, maxlen=100, padding='post')

value = model.predict(review_padded)


if value > 0.5:
    print("긍정적인 리뷰입니다.")
else:
    print("부정적인 리뷰입니다.")


1/1 [==============================] - 0s 19ms/step
부정적인 리뷰입니다.


In [9]:
# 긍정 부정 데이터 개수 확인
positive_count = ratings[ratings >= 3].count()
negative_count = ratings[ratings < 3].count()

print("긍정 데이터 개수:", positive_count)
print("부정 데이터 개수:", negative_count)

긍정 데이터 개수: 2755
부정 데이터 개수: 2414
